$$\textbf{Lab 4: Exercise 2}$$



$\textbf{Function under consideration: }\;\; q(\mathbf{x}) = \sqrt{x_1^2+4} + \sqrt{x_2^2+4}$

$\textbf{Gradient of the function:}$ \\
$\begin{aligned}\nabla q(\mathbf{x}) &= [\frac{\partial q(\mathbf{x})}{\partial x_1}, \frac{\partial q(\mathbf{x})}{\partial x_2}]^\top \\&= [\frac{x_1}{\sqrt{x_1^2+4}}, \frac{x_2}{\sqrt{x_2^2+4}}]^\top \end{aligned}$

$\textbf{Gradient of the function:}$ \\
$\begin{aligned}\nabla^2 q(\mathbf{x}) &= \begin{bmatrix} q_{x_1 x_1} & q_{x_1 x_2}\\ q_{x_2 x_1} & q_{x_2 x_2} \end{bmatrix}\\ &= \begin{bmatrix} \frac{4}{(x_1^2+4)^{\frac{3}{2}}} & 0\\ 0 & \frac{4}{(x_2^2+4)^{\frac{3}{2}}} \end{bmatrix}\end{aligned}$ 

where $q_{x_1 x_1} = \frac{\partial^2q(\mathbf{x})}{\partial x_1^2}, q_{x_1 x_2} = \frac{\partial^2q(\mathbf{x})}{\partial x_1 \partial x_2},  q_{x_2 x_1} = \frac{\partial^2q(\mathbf{x})}{\partial x_2 \partial x_1}, q_{x_2 x_2} = \frac{\partial^2q(\mathbf{x})}{\partial x_2^2}$


$\text{For Newtown's method, the scaling matrix $\mathbf{D}^k$  is the inverse of the Hessian matrix $\nabla^2 q(\mathbf{x})$.}$
$\text{For the gradient descent algorithm (with scaling using a diagonal matrix) with backtracking line search, the scaling matrix $\mathbf{D}^k$ is the diagonal approximation of the inverse of $\nabla^2 q(\mathbf{x})$, given as:}$ $\begin{aligned}
\mathbf{D}^k = \begin{bmatrix} \frac{1}{q_{x_1 x_1}} & 0\\ 0 & \frac{1}{q_{x_2 x_2}} \end{bmatrix}
\end{aligned}
$, where $f_{x_1 x_1} = \frac{\partial^2q(\mathbf{x})}{\partial x_1^2}$ and  $q_{x_2 x_2} = \frac{\partial^2q(\mathbf{x})}{\partial x_2^2}$

In [89]:
#Importing the packages
import numpy as np 
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt

In [90]:
#function that compute and return the function value 
def evalf(x):  
    #Input: x is a numpy array of size 2 
    assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments    
    return (np.sqrt((x[0]**2)+4)+np.sqrt((x[1]**2)+4))

In [91]:
#function that compute and return the gradient value as a numpy array 
def evalg(x):  
    #Input: x is a numpy array of size 2 
    assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments   
    return np.array([(x[0])/(np.sqrt((x[0]**2)+4)),  (x[1])/(np.sqrt((x[1]**2)+4))]) 

In [92]:
# Method to find Hessian matrix
def evalh(x):
    assert type(x) is np.ndarray 
    assert len(x) == 2
    elem1 = 4/((np.sqrt((x[0]**2)+4))*((x[0]**2)+4))
    elem2 = 4/((np.sqrt((x[1]**2)+4))*((x[1]**2)+4))
    return np.array([[elem1, 0], [0, elem2]])

In [93]:
# Method to construct D_k matrix used in GD with scaling with backtracking line search 
def compute_D_k(x):
    assert type(x) is np.ndarray
    assert len(x) == 2  
    elem_1 = evalh(x)[0][0]
    elem_2 = evalh(x)[1][1]
    return np.diag([1/elem_1,1/elem_2])

In [94]:
# Method to construct D_k matrix used in Newton's method
def compute_D_k_newton(x):
    assert type(x) is np.ndarray
    assert len(x) == 2   
    return np.linalg.inv(evalh(x))

In [95]:
# Module to compute the steplength by using the closed-form expression
def compute_steplength_exact(x, gradf, A, b): #add appropriate arguments to the function 
    b = b.reshape(b.shape[0],1)
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(A) is np.ndarray and A.shape[0] == 2 and  A.shape[1] == 2 #allow only a 2x2 array
    assert type(b) is np.ndarray and b.shape[0] == 2 and b.shape[1] == 1
    numerator = np.dot(np.dot(x.T, A), gradf) + np.dot(np.dot(gradf.T, A), x) + 2*np.dot(b.T, gradf)
    denominator = 2*np.dot(np.dot(gradf.T, A), gradf)
    step_length = numerator/denominator  
    return step_length

In [96]:
# Module to compute the steplength by using the closed-form expression
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): 
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*p)) > np.add(evalf(x), gamma*alpha*np.dot(gradf.T,p)):
        alpha = rho*alpha
    return alpha

In [97]:
def compute_steplength_backtracking_scaled_direction(x, gradf, D_k, alpha_start, rho, gamma):  
    assert type(x) is np.ndarray and len(gradf) == 2 
    assert type(gradf) is np.ndarray and len(gradf) == 2 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == 2 
    alpha = alpha_start
    p = -gradf
    while evalf(np.add(x, alpha*np.dot(D_k,p))) > np.subtract(evalf(x), gamma*alpha*np.dot(np.dot(D_k,gradf), gradf)):
        alpha = rho*alpha
    return alpha

In [98]:
#line search type 
CONSTANT_STEP_LENGTH = 1
BACKTRACKING_LINE_SEARCH = 2
# D_k choice type
newton = 3
diagonal_approx = 4

In [99]:
# Code for gradient descent with scaling to find the minimizer without scaling
def find_minimizer_gd(start_x, tol, line_search_type, *args): 
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg(x)
    #initialization for backtracking line search
    if(line_search_type == BACKTRACKING_LINE_SEARCH):
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]
    k = 0
    while (np.linalg.norm(g_x) > tol): 
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')   
        # Gradient descent steps   
        x = np.subtract(x, np.multiply(step_length,g_x))
        k += 1 
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x, k, evalf(x) 

In [113]:
# Code for gradient descent with scaling to find the minimizer with scaling
def find_minimizer_gdscaling(start_x, tol, line_search_type, D_k_type, *args):
    assert type(start_x) is np.ndarray and len(start_x) == 2 
    assert type(tol) is float and tol>=0 
    x = start_x
    g_x = evalg(x)   
    #initialization for backtracking line search
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    k = 0
    while (np.linalg.norm(g_x) > tol): 
        if D_k_type == newton:
            d_k = compute_D_k_newton(x)
        elif D_k_type == diagonal_approx:
            d_k = compute_D_k(x)
        else:
            raise ValueError("Invalid argument.")
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction(x, g_x, d_k, alpha_start, rho, gamma) 
        elif line_search_type == CONSTANT_STEP_LENGTH:
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')    
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x)))
        k += 1
        g_x = evalg(x) 
        #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
    return x, k, evalf(x)

#Q-2

In [104]:
my_start_x = np.array([2.0,2.0])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [102]:
#Gradient descent using Newton's method with constant step length
#x_opt_newton_const, iter_newton_const, f_opt_newton_const = find_minimizer_gdscaling(my_start_x, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
#print(f"\nNewton's method (with constant step length of 1) for starting point {my_start_x}:\nMinimizer: {x_opt_newton_const}\nFinal Objective function value: {f_opt_newton_const}\nIterations taken to terminate: {iter_newton_const}")

Streaming output truncated to the last 5000 lines.
iter: 31590  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 31591  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 31592  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 31593  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 31594  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 31595  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradient norm: 0.9999999999999999
iter: 31596  x: [2. 2.]  f(x): 5.656854249492381  grad at x: [0.70710678 0.70710678]  gradient norm: 0.9999999999999999
iter: 31597  x: [-2. -2.]  f(x): 5.656854249492381  grad at x: [-0.70710678 -0.70710678]  gradien

KeyboardInterrupt: ignored

In [105]:
# Gradient descent using Newton's method with backtracking line search
x_opt_newton_bls, iter_newton_bls, f_opt_newton_bls = find_minimizer_gdscaling(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print(f"\nNewton's method (with Backtracking line search) for starting point {my_start_x}:\nMinimizer: {x_opt_newton_bls}\nFinal Objective function value: {f_opt_newton_bls}\nIterations taken to terminate: {iter_newton_bls}")


Newton's method (with Backtracking line search) for starting point [2. 2.]:
Minimizer: [0. 0.]
Final Objective function value: 4.0
Iterations taken to terminate: 1


#*Comment:*
$\textbf{Regarding the number of iterations to converge:}
\; \; \text{ The Newton's method with constant step length does not converge. The minimizer keeps oscillating between $(2,2)$ and $(-2,-2)$ from the very first iteration.}
\; \; \text{ The Newton's method with backtracking line search requires only 1 iteration to terminate.} \\
\textbf{Regarding the obtained minimizer:}
\; \; \text{ The Newton's method with constant step length did not converge and we didn't obtain any minimizer for the function. The algorithm kept oscillating between $(2,2)$ and $(-2,-2)$.}
\; \; \text{ The Newton's method with backtracking line search yields the exact minimizer (0,0).} \\
\textbf{Regarding the function value at the minimizer:}
\; \; \text{ We didn't obtain any minimum function value using Newton's method with constant step length.}
\; \; \text{ The minimum objective function value at the obtained minimizer using Newton's method with backtracking line search is 4.}$


#Q-3

In [106]:
# Gradient descent with backtracking line search without scaling
x_opt_bls, iter_bls, f_opt_bls = find_minimizer_gd(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print(f"\nGradient Descent (without scaling) with backtracking line search for starting point {my_start_x}:\nMinimizer: {x_opt_bls}\nFinal Objective function value: {f_opt_bls}\nIterations taken to terminate: {iter_bls}")



Gradient Descent (without scaling) with backtracking line search for starting point [2. 2.]:
Minimizer: [7.62525638e-10 7.62525638e-10]
Final Objective function value: 4.0
Iterations taken to terminate: 32


#*Comment:*
$\textbf{Starting point = (2,2)} \\
\text{The Newton's method with constant step length did not converge. So, we do not keep it in the following comparing comments.} \\
\textbf{Regarding the number of iterations taken to converge:}
\; \; \text{ The gradient descent algorithm (without scaling) with backtracking line search terminates in 32 iterations,} \\
\; \; \text{higher than that required in the Newton's method with backtracking line search (which required 1 iteration).} \\
\textbf{Regarding the minimizer obtained:}
\; \; \text{ The minimizer obtained from the gradient descent algorithm (without scaling) with backtracking line search is (7.62525638e-10, 7.62525638e-10), which approximates to (0,0).} \\
\; \; \text{The Newton's method with backtracking line search yields the minimizer exactly as (0,0).}\\
\textbf{Regarding the minimum objective function value:}
\; \; \text{ The objective function value at the obtained minimizer in gradient descent algorithm (without scaling) with backtracking line search is 4,} \\
\; \; \text{which is same as the function value obtained from the Newton's method with backtracking line search.}
\; \; \text{ We can comment that appropriate selection of step length in every iteration is an important factor in the convergence of the algorithm. A constant step length does not yield required results.}$

#Q-4

In [114]:
my_start_x2 = np.array([8.0,8.0])
my_tol= 1e-9
alpha_start = 1.
rho = 0.5
gamma = 0.5

In [111]:
# Gradient descent using Newton's method with constant step length
#x_opt_newton_const2, iter_newton_const2, f_opt_newton_const2 = find_minimizer_gdscaling(my_start_x2, my_tol, CONSTANT_STEP_LENGTH, newton, alpha_start, rho, gamma)
#print(f"\nNewton's method (with constant step length of 1) for starting point {my_start_x2}:\nMinimizer: {x_opt_newton_const2}\nFinal Objective function value: {f_opt_newton_const2}\nIterations taken to terminate: {iter_newton_const2}")

iter: 1  x: [-128. -128.]  f(x): 256.03124809288414  grad at x: [-0.99987795 -0.99987795]  gradient norm: 1.414040960485301
iter: 2  x: [524288. 524288.]  f(x): 1048576.0000076294  grad at x: [1. 1.]  gradient norm: 1.4142135623628054
iter: 3  x: [-3.6028797e+16 -3.6028797e+16]  f(x): 7.205759403792794e+16  grad at x: [-1. -1.]  gradient norm: 1.4142135623730951
iter: 4  x: [1.16920131e+49 1.16920131e+49]  f(x): 2.3384026197294447e+49  grad at x: [1. 1.]  gradient norm: 1.4142135623730951
iter: 5  x: [-3.99583814e+146 -3.99583814e+146]  f(x): 7.99167628880894e+146  grad at x: [-1. -1.]  gradient norm: 1.4142135623730951


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  


LinAlgError: ignored

In [108]:
# Gradient descent using Newton's method with backtracking line search
x_opt_newton_bls2, iter_newton_bls2, f_opt_newton_bls2 = find_minimizer_gdscaling(my_start_x2, my_tol, BACKTRACKING_LINE_SEARCH, newton, alpha_start, rho, gamma)
print(f"\nNewton's method (with Backtracking line search) for starting point {my_start_x2}:\nMinimizer: {x_opt_newton_bls2}\nFinal Objective function value: {f_opt_newton_bls2}\nIterations taken to terminate: {iter_newton_bls2}")


Newton's method (with Backtracking line search) for starting point [8. 8.]:
Minimizer: [2.83764947e-12 2.83764947e-12]
Final Objective function value: 4.0
Iterations taken to terminate: 13


#*Comment:*
$\text{Starting point: (8,8)} \\
\textbf{Regarding the number of iterations to converge:}
\; \; \text{The Newton's method with constant step length is not coverging taking infinite iterations, that's why it is commented out.}
\; \; \text{The Newton's method with backtracking line search requires 13 iteration to terminate.} \\
\textbf{Regarding the obtained minimizer:}
\; \; \text{Due to the above mentioned reason, we were unable to obtain the minimizer value for The Newton's method with constant step length.}
\; \; \text{The Newton's method with backtracking line search yields the exact minimizer (2.83764947e-12, 2.83764947e-12).} \\
\textbf{Regarding the function value at the minimizer:}
\; \; \text{We didn't obtain any minimum function value using Newton's method with constant step length. The value overflowed and came out to be $\infty$.}
\; \; \text{The minimum objective function value at the obtained minimizer using Newton's method with backtracking line search is 4.}$


#Q-5

In [115]:
# Gradient descent with backtracking line search without scaling
x_opt_bls2, iter_bls2, f_opt_bls2 = find_minimizer_gd(my_start_x2, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
print(f"\nGradient Descent (without scaling) with backtracking line search for starting point {my_start_x2}:\nMinimizer: {x_opt_bls2}\nFinal Objective function value: {f_opt_bls2}\nIterations taken to terminate: {iter_bls2}")



Gradient Descent (without scaling) with backtracking line search for starting point [8. 8.]:
Minimizer: [8.3177047e-10 8.3177047e-10]
Final Objective function value: 4.0
Iterations taken to terminate: 39


#*Comment:*
$\textbf{Starting point = (8,8)} \\
\; \; \text{The Newton's method with constant step length did not converge. So, we do not keep it in the following comparing comments.} \\
\textbf{Regarding the number of iterations taken to converge:}
\; \; \text{The gradient descent algorithm (without scaling) with backtracking line search terminates in 39 iterations, higher than that required in the Newton's method with backtracking line search (which required 13 iterations).} \\
\textbf{Regarding the minimizer obtained:}
\; \; \text{The minimizer obtained from the gradient descent algorithm (without scaling) with backtracking line search is (8.3177047e-10 8.3177047e-10), which approximates to (0,0)}
\; \; \text{The Newton's method with backtracking line search yields the minimizer as (2.83764947e-12, 2.83764947e-12), which also approximates to (0,0). So both of them are comparable.} \\
\textbf{Regarding the minimum objective function value:}
\; \; \text{The objective function value at the obtained minimizer in gradient descent algorithm (without scaling) with backtracking line search is 4, which is same as the function value obtained from the Newton's method with backtracking line search.}
\; \; \text{For this particular function and choice of initial starting point,}
\; \; \text{ the gradient descent algorithm with scaling doesn't perform better than the gradient descent without scaling.}$